## Лабораторна робота №3
Виконав студент групи ФІ-31 Дремко Олександр

In [1]:
from spyre import server
from datetime import datetime, timedelta
import pandas as pd
import os
from urllib import request
from shutil import rmtree
import matplotlib.pyplot as plt
import __main__

1. Для кожної із адміністративних одиниць України завантажити тестові
структуровані файли, що містять значення VHI-індексу. Ця процедура має бути
автоматизована, параметром процедури має бути індекс (номер) області. При зберіганні файлу до його імені потрібно додати дату та час завантаження. Передбачити повторні запуски скрипту, довантаження нових даних та колізію даних

In [2]:
# Перевірка папки на порожність та очищення
def check_folder(folder_path):
    # Якщо папки не існує то створюємо її
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        
    # Якщо папка існує і не порожня то очищаємо її
    elif os.listdir(folder_path):
        rmtree(folder_path)
        os.makedirs(folder_path)

# Завантаження даних для області за id
def download_data(province_id):
    url = f'https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?country=UKR&provinceID={province_id}&year1=1981&year2=2024&type=Mean'
    try:
        vhi = request.urlopen(url).read().decode('utf-8')
        now = datetime.now()
        formatted_time = now.strftime('%d%m%Y%H%M%S')
        file_name = f'data/vhi_id_{province_id}_{formatted_time}.csv'
        with open(file_name, 'w') as f:
            f.write(vhi)
        print(f'Data for province {province_id} saved as {file_name}')
    except Exception as e:
        print(f'Error loading file for province {province_id}: {e}')

In [3]:
folder_path = 'data'
# check_folder(folder_path)

# for province_id in range(1, 28):
#     download_data(province_id)

2. Зчитати завантажені текстові файли у фрейм. Імена стовбців фрейму мають бути змістовними та легкими для сприйняття (не повинно бути спеціалізованих символів, пробілів тощо). Ця задача має бути реалізована у вигляді окремої процедури, яка на вхід приймає шлях до директорії, в якій зберігаються файли

In [4]:
def create_df(folder_path):
    all_dfs = []  # Список для збереження всіх DataFrame

    # Обхід усіх файлів у папці
    for filename in os.listdir(folder_path):
        if filename.endswith('.csv'):
            file_path = os.path.join(folder_path, filename)
            
            # Витягуємо area з імені файлу, розділяючи підкресленням (_)
            try:
                area = int(filename.split('_')[2])
            except (IndexError, ValueError):
                print(f"Не вдалося витягти area з файлу: {filename}")
                continue

            # Завантаження та очищення даних
            try:
                # Пропускаємо метадані та зчитуємо дані починаючи з заголовка
                df = pd.read_csv(file_path, skiprows=2, skipinitialspace=True)
                
                # Видаляємо порожні стовпці, якщо вони є
                df = df.dropna(axis=1, how="all")
                
                # Виправляємо назви стовпців
                df.columns = ['Year', 'Week', 'SMN', 'SMT', 'VCI', 'TCI', 'VHI']
                
                # Додаємо ідентифікатор області
                df['area'] = area
                
                # Перетворюємо дані на числові типи
                for col in ['Year', 'Week', 'SMN', 'SMT', 'VCI', 'TCI', 'VHI']:
                    df[col] = pd.to_numeric(df[col], errors='coerce')
                
                # Видаляємо рядки з NaN у стовпці Year
                df = df[df['Year'].notna()]

                # Видаляємо рядки з -1
                df = df[df['VHI'] != -1] 
                
                # Додаємо очищений DataFrame до списку
                all_dfs.append(df)
                
            except pd.errors.EmptyDataError:
                print(f"Файл порожній або не вдалося прочитати: {filename}")

    

    # Об’єднання всіх DataFrame в один
    if all_dfs:
        combined_df = pd.concat(all_dfs, ignore_index=True)
    else:
        print("Не вдалося об'єднати DataFrame: список порожній")
        return pd.DataFrame()  # Повертаємо порожній DataFrame у разі помилки

    # Виводимо інформацію про об'єднаний DataFrame
    print("Об'єднаний DataFrame створено")

    return combined_df

In [7]:
def convert_YearWeek_to_Date(df):
    # Перетворюємо значення стовпців Year і Week в int
    df['Year'] = df['Year'].astype(int)
    df['Week'] = df['Week'].astype(int)

    # Перетворюємо Year і Week у дату
    df['FormattedDate'] = df.Year * 1000 + df.Week * 10 + 0
    # df['FormattedDate'] = f"{df['Year']:04d}{df['Week']:02d}"
    df['Date'] = pd.to_datetime(df['FormattedDate'], format='%Y%W%w')

    # Сортуємо стовпці, видаляючи зайві
    columns_order = ['Date','Year', 'Week', 'SMN', 'SMT', 'VCI', 'TCI', 'VHI', 'area']
    df = df[columns_order]

    return df

In [8]:
df = create_df(folder_path)
print(df.head())

df = convert_YearWeek_to_Date(df)
print(df.head())

# print(df.dtypes)

# filtered_df = df[df['area'] == 1]
# print(filtered_df.head())

Об'єднаний DataFrame створено
     Year  Week    SMN     SMT    VCI    TCI    VHI  area
0  1982.0   2.0  0.063  261.53  55.89  38.20  47.04    10
1  1982.0   3.0  0.063  263.45  57.30  32.69  44.99    10
2  1982.0   4.0  0.061  265.10  53.96  28.62  41.29    10
3  1982.0   5.0  0.058  266.42  46.87  28.57  37.72    10
4  1982.0   6.0  0.056  267.47  39.55  30.27  34.91    10
        Date  Year  Week    SMN     SMT    VCI    TCI    VHI  area
0 1982-01-17  1982     2  0.063  261.53  55.89  38.20  47.04    10
1 1982-01-24  1982     3  0.063  263.45  57.30  32.69  44.99    10
2 1982-01-31  1982     4  0.061  265.10  53.96  28.62  41.29    10
3 1982-02-07  1982     5  0.058  266.42  46.87  28.57  37.72    10
4 1982-02-14  1982     6  0.056  267.47  39.55  30.27  34.91    10


3. Створити веб додаток за допомогою бібліотеки Spyre
- Додати випадаючий список який має містити VCI, TCI, VHI
- Додати випадаючий список для вибору області
- Додати текстові поля для введення інтервалу дат
- Вивести таблицю і графік з відсортованими данними

In [ ]:
__main__.__file__ = os.getcwd()

class NOAADataApp(server.App):
    title = "NOAA Data Visualization"

    inputs = [
        {
            "type": 'dropdown',  # VCI, TCI, VHI
            "label": 'Select Index',
            "options": [
                {"label": "VCI", "value": "VCI"},
                {"label": "TCI", "value": "TCI"},
                {"label": "VHI", "value": "VHI"}
            ],
            "key": "index",
            "action_id": "update_data"
        },
        {
            "type": 'dropdown',  # Вибір регіону
            "label": 'Select Region',
            "options": [{"label": f"Region {i}", "value": i} for i in sorted(df['area'].unique())],
            "key": "region",
            "action_id": "update_data"
        },
        {
            "type": 'text',  # Поле для введення початкової дати
            "key": 'start_date',
            "label": 'Enter Start Date',
            "value": '1982-01-01',
            "action_id": 'update_data'
        },
        {
            "type": 'text',  # Поле для введення кінцевої дати
            "key": 'end_date',
            "label": 'Enter End Date',
            "value": '1982-12-31',
            "action_id": 'update_data'
        }
    ]

    outputs = [
        {"type": "plot", "id": "plot", "control_id": "update_data", "tab": "Plot"},  # Графік
        {"type": "table", "id": "table_id", "control_id": "update_data", "tab": "Table"}  # Таблиця
    ]

    tabs = ["Table", "Plot"]  # Вкладки для таблиці та графіка

    # Функція для підготовки таблиці даних
    def getData(self, params):
        # Отримуємо параметри
        index = params['index']  # Вибраний індекс (VCI, TCI, VHI)
        region = int(params['region'])  # Вибрана область
        start_date = pd.to_datetime(params['start_date'])  # Початкова дата
        end_date = pd.to_datetime(params['end_date'])  # Кінцева дата

        # Фільтруємо дані за областю та діапазоном дат
        filtered_df = df[(df['area'] == region) & 
                         (df['Date'] >= start_date) & 
                         (df['Date'] <= end_date)]

        # Повертаємо тільки потрібні стовпці
        return filtered_df[['Year', 'Week', index]]

    # Функція для створення графіка
    def getPlot(self, params):
        # Використовуємо getData для отримання відфільтрованих даних
        filtered_df = self.getData(params)

        # Створюємо графік
        grouped = filtered_df.groupby('Year')
        plt.figure(figsize=(15, 6))
        for year, data in grouped:
            plt.plot(data['Week'], data[params['index']], marker='o', label=f'{year}')

        plt.title(f"{params['index']} по тижнях для регіону {params['region']}")
        plt.xlabel('Тиждень')
        plt.ylabel(params['index'])
        plt.legend(title='Рік')
        plt.grid(True)
        
        fig = plt.gcf()  # Отримуємо об'єкт фігури для відображення
        return fig

# Запуск додатку
if __name__ == "__main__":
    app = NOAADataApp()
    app.launch(port=9093)

[25/Nov/2024:14:45:17] ENGINE Listening for SIGTERM.
INFO:cherrypy.error:[25/Nov/2024:14:45:17] ENGINE Listening for SIGTERM.
[25/Nov/2024:14:45:17] ENGINE Bus STARTING
INFO:cherrypy.error:[25/Nov/2024:14:45:17] ENGINE Bus STARTING
[25/Nov/2024:14:45:17] ENGINE Set handler for console events.
INFO:cherrypy.error:[25/Nov/2024:14:45:17] ENGINE Set handler for console events.
CherryPy Checker:
The Application mounted at '' has an empty config.

[25/Nov/2024:14:45:17] ENGINE Started monitor thread 'Autoreloader'.
INFO:cherrypy.error:[25/Nov/2024:14:45:17] ENGINE Started monitor thread 'Autoreloader'.
[25/Nov/2024:14:45:18] ENGINE Serving on http://127.0.0.1:9093
INFO:cherrypy.error:[25/Nov/2024:14:45:18] ENGINE Serving on http://127.0.0.1:9093
[25/Nov/2024:14:45:18] ENGINE Bus STARTED
INFO:cherrypy.error:[25/Nov/2024:14:45:18] ENGINE Bus STARTED


127.0.0.1 - - [25/Nov/2024:14:45:26] "GET / HTTP/1.1" 200 442105 "" "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:132.0) Gecko/20100101 Firefox/132.0"


INFO:cherrypy.access.2079491014416:127.0.0.1 - - [25/Nov/2024:14:45:26] "GET / HTTP/1.1" 200 442105 "" "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:132.0) Gecko/20100101 Firefox/132.0"


127.0.0.1 - - [25/Nov/2024:14:45:26] "GET /table?index=VCI&region=1&start_date=1982-01-01&end_date=1982-12-31&output_id=table_id& HTTP/1.1" 200 4201 "http://127.0.0.1:9093/" "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:132.0) Gecko/20100101 Firefox/132.0"


INFO:cherrypy.access.2079491014416:127.0.0.1 - - [25/Nov/2024:14:45:26] "GET /table?index=VCI&region=1&start_date=1982-01-01&end_date=1982-12-31&output_id=table_id& HTTP/1.1" 200 4201 "http://127.0.0.1:9093/" "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:132.0) Gecko/20100101 Firefox/132.0"


...

INFO:cherrypy.access.2079491014416:127.0.0.1 - - [25/Nov/2024:14:45:46] "GET /spinning_wheel HTTP/1.1" 200 2663 "http://127.0.0.1:9093/" "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:132.0) Gecko/20100101 Firefox/132.0"


127.0.0.1 - - [25/Nov/2024:14:45:46] "GET /favicon.ico HTTP/1.1" 200 1406 "http://127.0.0.1:9093/" "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:132.0) Gecko/20100101 Firefox/132.0"


INFO:cherrypy.access.2079491014416:127.0.0.1 - - [25/Nov/2024:14:45:46] "GET /favicon.ico HTTP/1.1" 200 1406 "http://127.0.0.1:9093/" "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:132.0) Gecko/20100101 Firefox/132.0"
ERROR:root:Error: getPlot method must return an pyplot figure or matplotlib Axes object


127.0.0.1 - - [25/Nov/2024:14:45:46] "GET /plot?index=VHI&region=1&start_date=1982-01-01&end_date=1985-12-31&output_id=plot& HTTP/1.1" 200 95210 "http://127.0.0.1:9093/" "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:132.0) Gecko/20100101 Firefox/132.0"


INFO:cherrypy.access.2079491014416:127.0.0.1 - - [25/Nov/2024:14:45:46] "GET /plot?index=VHI&region=1&start_date=1982-01-01&end_date=1985-12-31&output_id=plot& HTTP/1.1" 200 95210 "http://127.0.0.1:9093/" "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:132.0) Gecko/20100101 Firefox/132.0"
